# Libraries

In [1]:
import nltk
import os
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random
import tensorflow as tf
import tkinter
from tkinter import *
from keras.models import load_model

Using TensorFlow backend.


# Extracting the data

In [2]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

# Tokenizing the sentence

In [3]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        
        w = nltk.word_tokenize(pattern)
        words.extend(w)
       
        documents.append((w, intent['tag']))
        
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# Lemmatization

In [4]:

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

47 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
88 unique lemmatized words ["'s", ',', 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


In [5]:
print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

88 unique lemmatized words ["'s", ',', 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


# Training the model

In [7]:

training = []

output_empty = [0] * len(classes)

for doc in documents:
    
    bag = []
    
    pattern_words = doc[0]
    
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    
for w in words:
    bag.append(1) if w in pattern_words else bag.append(0)
    
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)
print('training created')


training created


In [8]:
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


# Model Buliding

In [10]:
model=Sequential()

W0107 16:03:16.184602  8580 deprecation_wrapper.py:119] From D:\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



In [11]:
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd=SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

W0107 16:03:19.027795  8580 deprecation_wrapper.py:119] From D:\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0107 16:03:20.585516  8580 deprecation_wrapper.py:119] From D:\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0107 16:03:20.675004  8580 deprecation_wrapper.py:119] From D:\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0107 16:03:20.797840  8580 deprecation.py:506] From D:\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate shoul

Epoch 1/200
88/88 [==============================] - 1s 14ms/step - loss: 0.8849 - acc: 0.8636
Epoch 2/200
88/88 [==============================] - 0s 530us/step - loss: 0.0155 - acc: 1.0000
Epoch 3/200
88/88 [==============================] - 0s 492us/step - loss: 0.0025 - acc: 1.0000
Epoch 4/200
88/88 [==============================] - 0s 321us/step - loss: 0.0011 - acc: 1.0000
Epoch 5/200
88/88 [==============================] - 0s 398us/step - loss: 0.0037 - acc: 1.0000
Epoch 6/200
88/88 [==============================] - 0s 376us/step - loss: 0.0039 - acc: 1.0000
Epoch 7/200
88/88 [==============================] - 0s 325us/step - loss: 0.0033 - acc: 1.0000
Epoch 8/200
88/88 [==============================] - 0s 334us/step - loss: 3.7244e-04 - acc: 1.0000
Epoch 9/200
88/88 [==============================] - 0s 411us/step - loss: 0.0043 - acc: 1.0000
Epoch 10/200
88/88 [==============================] - 0s 381us/step - loss: 5.4943e-04 - acc: 1.0000
Epoch 11/200
88/88 [============

88/88 [==============================] - 0s 327us/step - loss: 6.9305e-05 - acc: 1.0000
Epoch 84/200
88/88 [==============================] - 0s 332us/step - loss: 7.9035e-04 - acc: 1.0000
Epoch 85/200
88/88 [==============================] - 0s 333us/step - loss: 6.6066e-05 - acc: 1.0000
Epoch 86/200
88/88 [==============================] - 0s 326us/step - loss: 6.6336e-05 - acc: 1.0000
Epoch 87/200
88/88 [==============================] - 0s 366us/step - loss: 4.3401e-05 - acc: 1.0000
Epoch 88/200
88/88 [==============================] - 0s 342us/step - loss: 1.5061e-04 - acc: 1.0000
Epoch 89/200
88/88 [==============================] - 0s 297us/step - loss: 6.1993e-05 - acc: 1.0000
Epoch 90/200
88/88 [==============================] - 0s 376us/step - loss: 2.2973e-04 - acc: 1.0000
Epoch 91/200
88/88 [==============================] - 0s 335us/step - loss: 3.0523e-05 - acc: 1.0000
Epoch 92/200
88/88 [==============================] - 0s 407us/step - loss: 4.6121e-06 - acc: 1.0000
Epo

88/88 [==============================] - 0s 327us/step - loss: 4.4647e-06 - acc: 1.0000
Epoch 164/200
88/88 [==============================] - 0s 291us/step - loss: 3.6679e-05 - acc: 1.0000
Epoch 165/200
88/88 [==============================] - 0s 323us/step - loss: 4.4164e-05 - acc: 1.0000
Epoch 166/200
88/88 [==============================] - 0s 339us/step - loss: 1.9143e-05 - acc: 1.0000
Epoch 167/200
88/88 [==============================] - 0s 335us/step - loss: 3.5439e-05 - acc: 1.0000
Epoch 168/200
88/88 [==============================] - 0s 308us/step - loss: 9.7963e-05 - acc: 1.0000
Epoch 169/200
88/88 [==============================] - 0s 369us/step - loss: 1.2479e-04 - acc: 1.0000
Epoch 170/200
88/88 [==============================] - 0s 386us/step - loss: 2.7219e-05 - acc: 1.0000
Epoch 171/200
88/88 [==============================] - 0s 369us/step - loss: 1.1405e-05 - acc: 1.0000
Epoch 172/200
88/88 [==============================] - 0s 343us/step - loss: 1.2893e-05 - acc: 1

In [12]:
model.save('chat_bot.h5', hist)

In [13]:
len(train_y[0])

9

In [14]:
from keras.models import load_model

In [15]:
model=load_model('chat_bot.h5')
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [16]:
def clean_up_sentence(sentence):
    sentence_words=nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words



In [17]:
def bow(sentence, words, show_details=True):
   
    sentence_words = clean_up_sentence(sentence)
    
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

In [24]:
def predict_class(sentence, model):
   
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
   
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list


In [19]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result


In [20]:
def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res

In [48]:
def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)
    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))
        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)

    


In [ ]:
base = Tk()
base.title("Hello")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)
ChatLog.config(state=DISABLED)

scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

In [ ]:
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")

scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)
base.mainloop()